# 8 Applying Machine Learning to Sentiment Analysis

本章では、自然言語処理の一分野である感情分析を取り上げる<br>
そして、機械学習のアルゴリズムを使用することで、極性に基づいて文書を分類する方法を学ぶ<br>
極性とは、書き手の意見のことである<br>
本書では、IMDbの50000件の映画レビューで構築されたデータセットを操作し、肯定的または否定的なレビューを分類できる予測機を構築する<br>

・テキストデータのクレンジングと準備<br>
・テキスト文書からの特徴ベクトルの構築  
・映画レビューを肯定的な文と否定的な文に分類する機械学習のモデルのトレーニング  
・アウトオブコア学習に基づく大規模なテキストデータセットの処理  
・文書コレクションからカテゴリのトピックを推定する

# 8.1 IMDbの映画データセットでのテキスト処理

感情分析は、広大な分野であるNLPの一分野としてよく知られており、文書の極性を分析することに関連している  
感情分析は、意見マイニングとも呼ばれる  
感情分析でよく知られているタスクは、ある話題に関して書き手が表明した意見や感情に基づいて文書を分類することである  
映画レビューのサブセットから意味のある情報を抽出し、レビューした人が映画を「好き」と評価したのか、「嫌い」と評価したのかを予測できる機械学習モデルの構築方法について説明する

# 8.1.1 映画レビューデータセットを取得する

Pythonを使ってtarfileアーカイブを直接展開する

In [3]:
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

# 8.1.2 映画レビューデータセットをより便利なフォーマットに変換する

データセットを取り出したら、ダウンロードアーカイブに含まれていたテキスト文書を1つのCSVファイルにまとめる  
次のコードは、映画レビューをpandasのDataFrameオブジェクトに読み込む  
この処理には、標準的なデスクトップコンピュータで10分ほどかかることがある  
進行状況と推定時間を確認するにはPyPrindパッケージを使用する  
PyPrindをインストールするには、pip install pyprindコマンドを実行する  
インストールが完了したら、次のコードを実行できる

In [5]:
import pyprind
import pandas as pd
import os
# 'bashpath'の値を展開した映画レビューデータセットのディレクトリに置き換える
basepath = 'aclImdb'
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:04:01


このコードでは、まずプログレスオーバーオブジェクトpbarを50000回のいてレーションで初期化している  
この回数は読み込みの対象となる文書の個数に匹敵する  
入れ子のforループを使ってaclImdbディレクトリのtrainサブディレクトリとtestサブディレクトリを処理し、posサブディレクトリとnegサブディレクトリから個々のテキストファイルを読み込んでいる  
そして最後に、dfというDataFrameオブジェクトにそれらのファイルを整数のクラスラベルとともに追加している  
1のクラスラベルは「肯定的」、0のクラスラベルは「否定的」を表す  

データセットに組み込まれているクラスラベルはソート済みであるため、次のコードに示すように、np.randomサブモジュールのpermutation関数を使って行の順番をシャッフルしたDataFrameオブジェクトを作成する  

In [6]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

このデータセットは後程使用することになるため、データを正しいフォーマットで保存出来ていることを簡単に確認しておく  
CSVファイルを読み込み、最初の3つのサンプルから抜粋したデータを出力する

In [7]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


# 8.2 BoWモデルの紹介

文章や単語などのカテゴリーは、機械学習アルゴリズムに渡す前に数値に変換しておく必要がある  
ここでは、テキストを数値の特徴ベクトルとして表現できるBoWモデルを紹介する  
BoWもでるの背景にある考え方はとても単純で、次のように要約できる  

1.文書の集合全体から、たとえば単語という一意なトークンからなる語彙を作成する  
2.各文書での各単語の出現回数を含んだ特徴ベクトルを構築する

各文書において一意な単語は、BoWの語彙を構成しているすべての単語の一部に過ぎない  
このとき、特徴ベクトルの大半は0になるため、疎ベクトルと呼ばれる  

# 8.2.1 単語を特徴ベクトルに変換する

各文書に含まれる単語に基づいてBoWモデルを構築するには、scikit-learnに実装されているConutVectorizerクラスを使用できる  
次のコードに示すように、このクラスはテキストデータの配列を入力として、BoWモデルを自動的に生成する

In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

CountVectorizerクラスのfit_transformメソッドを呼び出すことで、BoWモデルの語彙を生成し、次の3つの文章を疎なベクトルに変換している

- 'The sun is shining'
- 'The weather is sweet'
- 'The sun is shining, the weather is sweet, and one and one is two'

In [10]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


このコマンドを実行するとわかるように、語彙はディクショナリに格納されている  
このディクショナリの要素は一意な単語と整数値を対応づけたものであり、たとえば'and'と0を対応付ける

In [11]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


この特徴ベクトルの各要素のインデックスは、ディクショナリの整数値に対応している  
例えば、整数値0に対応する最初の特徴量は、最後の文書にのみ出現する単語'and'の個数である  
整数値1に対応する単語'is'は、文書ベクトルの2つ目の特徴量であり、3つの文章のすべてに出現している  
特徴ベクトルにおけるそれらの値は生の出現頻度ともよばれ、tf(t,d)で表される  
これは文書dにおける単語tの出現回数を表す

# 8.2.2 TF-IDFを使って単語の関連性を評価する

テキストデータを解析していると、「肯定的」、「否定的」など両極のクラスそれぞれに分類される複数の文書において、同じ単語が出現することがよくある  
そうした頻繁に出現する単語は、たいてい、意味のある情報や判別情報を含んでいない  
ここでは、TF-IDFという便利な手法について説明する  
この手法を利用すれば、特徴ベクトルに頻繁に出現する単語の重みを減らすことができる  
TF-IDFは、TF（単語の出現頻度）とIDF（逆文書頻度）の積として定義できる

$$tf-idf(t,d) = tf(t,d)×idf(t,d)$$

ここで、tf(t,d)は、前項で説明した単語の出現頻度である  
逆文書頻度idf(t,d)は次の方法で求めることができる

$$idf(t,d) = log\frac{n_d}{1+df(t,d)}$$

$n_d$は文書の総数、df(t,d)は単語tを含んでいる文書dの個数を表す  
母数に定数1を足すのは、トレーニングサンプルに出現するすべての単語に0以外の値を割り当てることで、ゼロ割を回避するためである  
また、対数が使用されているのは、頻度の低い文書に過剰な重みが与えられないようにするためである

scilit-learnには、TfidfTransformerクラスという変換器も実装されている  
このクラスは、CountVectorizer（のfit_transformメソッド）から「生の単語の出現頻度」を入力として受け取り、それらをTF-IDFに変換する

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


単語'is'の出現頻度が最も高いのは3つ目の文書であり、再頻出単語となっている  
だが、単語'is'は文書１と文書２にも含まれている．したがって、有益な判別情報を含んでいるとは考えにくい  
そこで、前項の特徴ベクトルをTF-IDFに変換すれば、文書３において単語'is'がそれほど大きくないTF-IDF（0.45）に関連付けられることが確認できるはずだ  
対照的に、単語'one'は3つ目の文章で2回出現しており、TF-IDFはより大きい0.5である  
単語'one'は3つ目の文書だけに出現しており、より判別的な情報になっている

特徴ベクトルの単語のTF-IDFを手動で計算すれば、先に定義した教科書どおりの「標準的」な定義式と比べて、TfidfTransformerによるTF-IDFの計算が少し異なることがわかる  
まず、scikit-learnに実装されているIDFとTF-IDFの式は次の通り

$$idf(t,d) = log\frac{1+n_d}{1+df(t,d)}+1$$

同様に、scikit-learnで計算されるTD-IDFは、先に定義したデフォルトの式とは少し異なっている

$$tf-idf(t,d) = tf(t,d)×idf(t,d)$$

TF-IDFを計算する前に「生の単語の出現頻度」を正規化するのがより一般的である  
ただし、TfidTransformerはTF-IDFを直接正規化するため、この方法を検討してみる  
デフォルト（norm='l2'）では、scikit-learnのTfidfTransformerはL2正規化を適用する  
その場合は、正規化されていない特徴ベクトルvをL2正規化で割ることにより、長さ１のベクトルが返される

$$v_{norm} = \frac{v}{||v||_2} = \frac{v}{(\sum_{i=1}^{n}{v_{i}^{2}})^{1/2}}$$

TfidfTransformerの仕組みを理解していることを確認するため、例を追いかけながら、3つ目の文書に対して単語'is'のTF-IDFを計算してみる

単語'is'の3つ目の文書での出現頻度は3（TF=3）である  
この単語は3つの文書のすべてに出現するため、この単語の文書頻度は3（DF=3）である  
よって、IDFを次のように計算できる

$$idf("is",d_3) = log\frac{1+3}{1+3} = 0$$

TF-IDFを計算するには、IDFに1を足し、それにTFを掛ければよい

$$tf-idf("is",d_3) = 3×(0+1) = 3$$

この計算を3つ目の文書のすべての単語で繰り返すと、次のTF-IDFベクトル[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]が得られる  
ただし、この特徴ベクトルの値が先ほど使用したTfidfTransformerから取得した値とは異なることが分かる  
このTF-IDFの計算に足りない最後の手順は、L2正規化である

$$tf-idf("is",d_3)_(norm) = 0.45$$

このように、scikit-learnのTfidfTransformerから返された結果と一致することが分かる  
TF-IDFがどのように計算されるのかを理解したところで、次項では、このような考え方を映画レビューデータセットに適用してみることにしよう

# 8.2.3 テキストデータのクレンジング

ここまでの項では、BoWモデル、単語の出現頻度、TF-IDFについて説明した  
だが不要な文字をすべて取り除くことにより、テキストデータをクレンジングすることが最初の重要な手順となる  
これがなぜ重要なのかを説明するために、再びシャッフルした映画レビューデータセットの1つ目の文書から、最後の50文字を出力してみる

In [19]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

見てのとおり、このテキストにはHTMLマークアップに加えて、句読点やその他の非英字文字列が含まれている  
HTMLマークアップはそれほど重要な意味を含んでいないものの、文脈によっては、句読点は有益な追加情報を表すことがある  
ただし、ここでは話を単純にするために、感情分析に役立つ":)"のような顔文字だけを残し、それ以外の句読点はすべて削除する  
そのためpreprocessor関数を定義し、そこでPythonの正規表現ライブラリであるreを使用する

In [20]:
import re 
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', ''))
    return text

このコードでは、1つ目の正規表現<[^>] * >を使用することで、映画レビューに含まれているHTMLマークアップを完全に削除しようとしている  
多くのプログラマは概してHTMLの解析に正規表現を使用しないように勧めてはいるが、このデータセットなら、この正規表現で十分に「クレンジング」できるはずだ  
HTMLマークアップを削除した後、もう少し複雑な正規表現を使って顔文字を検索し、一時的にemoticonsとして格納している  
次に、正規表現[\W]+を使って単語の一部ではない文字を全て削除し、テキストを小文字に変換している

続いて、一時的に格納したemoticonsを処理済みの文書文字列の末尾に付け足している  
さらに、一貫性を保つために、顔文字から「鼻」文字（-）を削除している

クレンジングした文書文字列の末尾に顔文字を追加するのは、特に洗練された方法には思えないかもしれない  
だが、このBoWモデルでは、語彙を構成しているトークンが1つの単語に過ぎないしたら、単語の順序は重要ではない  
文書を個々の言葉や単語、トークンに分割する方法についてさらに説明する前に、先に定義したpreprocessor関数が正しく動作することを確認しておく

In [21]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [22]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :):(:)'

最後に、以降の説明では「クレンジングした」テキストデータを繰り返し使用することになる  
そのため、DataFrameオブジェクトに含まれているすべての映画レビューにpreprpcessor関数を適用しておこう

In [23]:
df['review'] = df['review'].apply(preprocessor)

# 8.2.4 文書をトークン化する